# Figure 2: Experiment 1, Neuroimaging Temporal Phase ($\sigma_{\phi}$),
This notebook is used to recreate Figure 2, showing the temporal phase [$\sigma_{\phi}$] results for the neuroimaging data in Experiment 1. The data acquired has the following dimensions:
 1. X = 128
 2. Y = 100
 3. Slices = 6
 4. Diffusion directions [b=0 + $G_x$, $G_y$, $G_z$] = 4
 5. Repetitions = 5
 6. Trigger Delays = 8

A user can selcet the volunteer, slice, and diffusion direction, and trigger-delay to then compute the temporal phase for a given grouping of images (Fig. 2A). The user can also select the slice and diffusion direction to plot the mean temporal phase variation across the 8 different trigger delay times (Fig. 2B). These are the same steps that are used to create Supplementary Figures 1 & 2. 

The last portion of the code shows heatmaps so a user can test signifiance between different different motion compensation levels for different trigger-delay times. 

In [ ]:
# Import utilities
